# Obtaining Temporary Credentials from IDT SciTools Plus

This notebook walks you through obtaining the temporary credentials needed to make API requests to SciTools Plus.

## Prerequisites

You'll need:
- **Client ID** and **Client Secret** from IDT

For instructions on obtaining these credentials, see [IDT's API documentation](https://www.idtdna.com/pages/tools/apidoc).

## ⚠️ Security Warning

After using this notebook:
1. Clear all outputs (click the dropdown arrow next to the "Run all" button and select "Clear all outputs")
2. Shut down the Colab runtime

This prevents exposure of your password and credentials.

In [ ]:
# @title Your IDT SciPlus Plus Account Information

# @markdown Enter the account information below and press "▷ Run all" button above (just below the menu bar).

IDT_username = "" #@param {type:"string"}
IDT_password = "" #@param {type:"string"}
IDT_client_id = "" #@param {type:"string"}
IDT_client_secret = "" #@param {type:"string"}

In [ ]:
# This code is from the IDT.

from base64 import b64encode
import json
from urllib import request, parse

def get_access_token(client_id, client_secret, idt_username, idt_password):
    """
    Create the HTTP request, transmit it, and then parse the response for the
    access token.

    The body_dict will also contain the fields "expires_in" that provides the
    time window the token is valid for (in seconds) and "token_type".
    """

    # Construct the HTTP request
    authorization_string = b64encode(bytes(client_id + ":" + client_secret, "utf-8")).decode()
    request_headers = { "Content-Type" : "application/x-www-form-urlencoded",
                        "Authorization" : "Basic " + authorization_string }

    data_dict = {   "grant_type" : "password",
                    "scope" : "test",
                    "username" : idt_username,
                    "password" : idt_password }
    request_data = parse.urlencode(data_dict).encode()

    post_request = request.Request("https://www.idtdna.com/Identityserver/connect/token",
                                    data = request_data,
                                    headers = request_headers,
                                    method = "POST")

    # Transmit the HTTP request and get HTTP response
    response = request.urlopen(post_request)

    # Process the HTTP response for the desired data
    body = response.read().decode()

    # Error and return the response from the endpoint if there was a problem
    if (response.status != 200):
        raise RuntimeError("Request failed with error code:" + response.status + "\nBody:\n" + body)

    body_dict = json.loads(body)
    return body_dict

response = get_access_token(IDT_client_id, IDT_client_secret, IDT_username, IDT_password)

if response.get('token_type') == 'Bearer':
  htmldata = f"""
      <font size="12pt">
      Your access token is:
      <input type="text" size="50" style="font-size: 24pt" value="{response.get('access_token')}">
      <p style="color: red; font-weight: bold">
        It will expire in {response.get('expires_in')} seconds.
      </p>
      </font>
    """

  from IPython.display import HTML, display
  display(HTML(htmldata))